In [0]:
 g%pip install -r ../requirements.txt
%pip install mosaicml

In [0]:
%restart_python

In [0]:
%run ../setup/00_setup

In [0]:
import mlflow
import os

HF_DATASETS_CACHE = "/Volumes/will_smith/datasets/imagenet_tiny"
os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE

username = spark.sql("SELECT current_user()").first()['current_user()']
username

experiment_path = f'/Users/{username}/composer-distributor'

# Retrieve workspace URL and API token using dbutils notebook commands
db_host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
db_token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Manually create the experiment so that you know the ID and can send that to the worker nodes when you are ready to scale
experiment = mlflow.set_experiment(experiment_path)

## Data Splits

#### Total Rows: 110,000 (0.35 gb)


| Split       | # of examples |
|-------------|---------------|
| Train       | 100,000  |
| Validation  | 10,000  |

In [0]:
from datasets import load_dataset
import datasets

datasets.utils.logging.disable_progress_bar()
loaded_dataset = load_dataset('zh-plus/tiny-imagenet', cache_dir=HF_DATASETS_CACHE, trust_remote_code=True)

In [0]:
def transforms(examples):
  examples["image"] = [image.convert("RGB").resize((64,64)) for image in examples["image"]]
  return examples

loaded_dataset = loaded_dataset.map(transforms, batched=True)

In [0]:
import logging
import torchvision.models as models
from composer.models import ComposerClassifier

batch_size = 128
num_epochs = "2ep"
num_class =len(set(loaded_dataset["train"]["label"]))

logging.basicConfig(level=logging.INFO)
resnet = models.resnet18(pretrained=False)
resnet_composer = ComposerClassifier(resnet, num_classes=num_class)

In [0]:
import torch 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from datasets import Dataset

train_dataset = loaded_dataset.get('train', None)
test_dataset = loaded_dataset.get('valid', None)

# Apply transformations directly to the dataset
train_dataset =  Dataset.from_dict({"image": train_dataset['image'], "label": train_dataset["label"]}).with_format("torch", device="cuda")
test_dataset =  Dataset.from_dict({"image": test_dataset['image'], "label": test_dataset["label"]}).with_format("torch", device="cuda")

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [0]:
import torchvision
import torch.nn.functional as F
from composer.models import ComposerModel

class ResNet50(ComposerModel):

    def __init__(self):
        super().__init__()
        self.model = torchvision.models.resnet50()

    def forward(self, batch): # batch is the output of the dataloader
        # specify how batches are passed through the model
        inputs, _ = batch
        return self.model(inputs)

    def loss(self, outputs, batch):
        # pass batches and `forward` outputs to the loss
        _, targets = batch
        return F.cross_entropy(outputs, targets)

In [0]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = datasets.ImageNet("data", train=True, download=True, transform=transform)
mnist_dataloader = DataLoader(dataset, batch_size=128)

In [0]:

from composer import Trainer
from composer.algorithms import LabelSmoothing, CutMix, ChannelsLast
import torch.optim as optim

model = ResNet18()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

trainer = Trainer(
    model=model,
    optimizers=optimizer,
    train_dataloader=train_dataloader,
    # train_dataloader=train_dataloader,
    # eval_dataloader=test_dataloader,
    max_duration=num_epochs,
    algorithms=[
        LabelSmoothing(smoothing=0.1),
        CutMix(alpha=1.0),
        ChannelsLast(),
        ]
)
trainer.fit()